## Qwen3-14B's Fine-Tuning Test

official
> ref: https://docs.unsloth.ai/models/qwen3-how-to-run-and-fine-tune#fine-tuning-qwen3-with-unsloth \
> ref: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb

note
> ref: https://note.com/sachi2222/n/na7b1d91ffb5d

In [1]:
# imports
## LLM
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer, SFTConfig

import torch
from transformers import TrainingArguments

## Dataset
from datasets import load_dataset, Dataset
from unsloth import to_sharegpt
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Load & Setting Model

In [4]:
# model loading
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length=40960, # context length
    load_in_4bit=True, # less mem mode
    load_in_8bit=False, # more mem mode
    full_finetuning=False # if u need full-FT
)

'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 73be3647-f5e3-4934-9913-45db62d5169a)')' thrown while requesting HEAD https://huggingface.co/unsloth/Qwen3-14B-unsloth-bnb-4bit/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: a6e87096-7bb2-4914-988c-c1329d28d519)')' thrown while requesting HEAD https://huggingface.co/unsloth/Qwen3-14B-unsloth-bnb-4bit/resolve/main/config.json
Retrying in 2s [Retry 2/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 2a586d62-55d5-4105-b112-5fff00c422eb)')' thrown while requesting HEAD https://huggingface.co/unsloth/Qwen3-14B-unsloth-bnb-4bit/resolve/main/config.json
Retrying in 4s [Retry 3/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 6a86066a-4782-499e-ba

==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.431 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# select LoRA option
model = FastLanguageModel.get_peft_model(
    model,
    r=32, # Choose any num like 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha=32, # alpha = r | r*2
    lora_dropout=0, # 0 is optimized
    bias="none", # "none" is optimized
    use_gradient_checkpointing="unsloth", # True | "unsloth", for too-long context
    random_state=3407,
    use_rslora=False, # under here, optical
    loftq_config=None
)

Unsloth 2025.9.11 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


## Convert Datasets

In [7]:
# make Datasets
# load json
raw_data = load_dataset("json", data_files="./rspec_rules.json")

# convert hugginface-style
huggingface_data = Dataset.from_list(raw_data['train'])

# convert share_gpt-style
# Three dialogue sessions are preferred
share_data = to_sharegpt(huggingface_data,
                        merged_prompt="{title}",
                        merged_column_name="title",
                        output_column_name="description",
                        conversation_extension=3)


Merging columns:   0%|          | 0/273 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/273 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/273 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/273 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/273 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/273 [00:00<?, ? examples/s]

In [8]:
# Minor format changes
converted_share_data = [[{
            "role": "user" if message["from"] == "human" else "assistant",
            "content": re.sub(r"\('(.+?)',\)", r"\1", message["value"])
        }
        for message in item["conversations"]
    ]
    for item in share_data
]

In [9]:
converted_share_data[0]

[{'role': 'user',
  'content': 'Regex patterns following a possessive quantifier should not always fail'},
 {'role': 'assistant',
  'content': 'Possessive quantifiers in Regex patterns like below improve performance by eliminating needless backtracking: ?+ , *+ , ++ , {n}+ , {n,}+ , {n,m}+ But because possessive quantifiers do not keep backtracking positions and never give back, the following sub-patterns should not match only similar characters. Otherwise, possessive quantifiers consume all characters that could have matched the following sub-patterns and nothing remains for the following sub-patterns. Noncompliant code example "/a++abc/" // Noncompliant, the second \'a\' never matches "/\\d*+[02468]/" // Noncompliant, the sub-pattern "[02468]" never matches Compliant solution "/aa++bc/" // Compliant, for example it can match "aaaabc" "/\\d*+(?&lt;=[02468])/" // Compliant, for example it can match an even number like "1234"'},
 {'role': 'user',
  'content': 'A subclass should not be i

In [10]:
# Addition of <\llm_start\>, etc
conversations = tokenizer.apply_chat_template(
    converted_share_data,
    tokenize = False,
)

In [11]:
# Converted to Hugging Face format

targetdataset = Dataset.from_dict({"text": conversations})

## Train settings

In [13]:
# Train
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=targetdataset,
    eval_dataset=None, # test Datasets
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs=2,
        max_steps=60,
        learning_rate=4e-5,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/273 [00:00<?, ? examples/s]

## Start FT

In [ ]:
trainer_stats = trainer.train()

## Inference

In [15]:
# Inference test
prompt="""
Please provide examples of vulnerabilities related to external control of filenames or pathnames.
"""

messages = [{"role" : "user", "content" : prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2048, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)


In the context of software development, vulnerabilities related to external control of filenames or pathnames occur when an application uses user-supplied input to construct filenames or paths without proper validation or sanitization. This can lead to several security issues, including:

1. **Path Traversal Vulnerabilities**: Attackers can manipulate the input to access files or directories outside the intended scope, potentially leading to unauthorized access to sensitive files or system resources.

2. **Directory Traversal Vulnerabilities**: Similar to path traversal, this involves manipulating the input to navigate through directory structures, which can result in unauthorized access to files or directories that should be restricted.

3. **File Inclusion Vulnerabilities**: If an application includes files based on user input without proper validation, attackers can include malicious files, potentially leading to code execution or data leakage.

4. **File Upload Vulnerabilities**: W

In [31]:
# Inference test2
prompt="""
You are a white hat hacker tasked with discovering vulnerabilities in the provided source code.
Perform the following three actions on the source code below:
1. Identify the vulnerability
2. Present the risks of leaving it unaddressed
3. Provide a solution to eliminate the vulnerability

Ensure your output adheres to the following three points:
1. Output in Japanese
2. Be clear and concise
3. Use Markdown format

---
<?php
// create.php
require 'db.php';

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    $title = $_POST['title'];
    $end_time = $_POST['end_time'];
    $options = $_POST['options']; // 配列として受け取る

    // バリデーション（空欄チェック）
    $valid_options = array_filter($options, function($v) { return trim($v) !== ''; });
    
    if ($title && $end_time && count($valid_options) >= 2) {
        // トランザクション開始
        $pdo->beginTransaction();
        try {
            // スレッド保存
            $stmt = $pdo->prepare("INSERT INTO polls (title, end_time) VALUES (?, ?)");
            $stmt->execute([$title, $end_time]);
            $poll_id = $pdo->lastInsertId();

            // 選択肢保存
            $stmt_opt = $pdo->prepare("INSERT INTO options (poll_id, name) VALUES (?, ?)");
            foreach ($valid_options as $opt_name) {
                $stmt_opt->execute([$poll_id, $opt_name]);
            }

            $pdo->commit();
            header("Location: index.php"); // メインへリダイレクト
            exit;
        } catch (Exception $e) {
            $pdo->rollBack();
            $error = "作成に失敗しました。";
        }
    } else {
        $error = "タイトル、終了時間、および2つ以上の選択肢を入力してください。";
    }
}
?>
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <title>スレッド作成</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="header">
        <h1>新規スレッド作成</h1>
    </div>

    <?php if (isset($error)): ?>
        <p style="color: red;"><?= h($error) ?></p>
    <?php endif; ?>

    <form method="post" id="createForm">
        <div class="form-group">
            <label>タイトル</label>
            <input type="text" name="title" class="form-control" required placeholder="例: 好きなプログラミング言語は？">
        </div>

        <div class="form-group">
            <label>終了時間</label>
            <input type="datetime-local" name="end_time" class="form-control" required>
        </div>

        <div class="form-group">
            <label>選択肢 (2つ以上必須)</label>
            <div id="options-container">
                <input type="text" name="options[]" class="form-control" placeholder="選択肢 1" required style="margin-bottom:5px;">
                <input type="text" name="options[]" class="form-control" placeholder="選択肢 2" required style="margin-bottom:5px;">
            </div>
            <div class="add-option" onclick="addOption()">＋ 選択肢を追加する</div>
        </div>

        <div style="margin-top: 20px;">
            <button type="submit" class="btn btn-primary">作成する</button>
            <a href="index.php" class="btn btn-secondary">キャンセル</a>
        </div>
    </form>

    <script>
        function addOption() {
            const container = document.getElementById('options-container');
            const input = document.createElement('input');
            input.type = 'text';
            input.name = 'options[]';
            input.className = 'form-control';
            input.placeholder = '選択肢 ' + (container.children.length + 1);
            input.style.marginBottom = '5px';
            input.required = true;
            container.appendChild(input);
        }
    </script>
</body>
</html>

---
<?php
// db.php
try {
    // データベースファイルを作成/接続
    $pdo = new PDO('sqlite:voting_app.db');
    $pdo->setAttribute(PDO::ATTR_ERRMODE, PDO::ERRMODE_EXCEPTION);

    // スレッド（polls）テーブル作成
    $pdo->exec("CREATE TABLE IF NOT EXISTS polls (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        end_time DATETIME NOT NULL,
        created_at DATETIME DEFAULT CURRENT_TIMESTAMP
    )");

    // 選択肢（options）テーブル作成
    $pdo->exec("CREATE TABLE IF NOT EXISTS options (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        poll_id INTEGER,
        name TEXT NOT NULL,
        votes INTEGER DEFAULT 0,
        FOREIGN KEY (poll_id) REFERENCES polls(id)
    )");

} catch (PDOException $e) {
    die("データベース接続エラー: " . $e->getMessage());
}

// XSS対策用関数
function h($str) {
    return htmlspecialchars($str, ENT_QUOTES, 'UTF-8');
}
?>

---
<?php
// index.php
require 'db.php';

// スレッド一覧を取得（終了時間が新しい順）
$stmt = $pdo->query("SELECT * FROM polls ORDER BY created_at DESC");
$polls = $stmt->fetchAll(PDO::FETCH_ASSOC);
?>
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <title>投票アプリ - メイン</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="header">
        <h1>スレッド一覧</h1>
        <a href="create.php" class="btn btn-primary">＋ スレッド作成</a>
    </div>

    <div class="poll-list">
        <?php if (count($polls) === 0): ?>
            <p>現在スレッドはありません。</p>
        <?php else: ?>
            <?php foreach ($polls as $poll): ?>
                <?php 
                    $is_expired = new DateTime() > new DateTime($poll['end_time']);
                    $status_text = $is_expired ? '<span style="color:red">【終了】</span>' : '<span style="color:green">【受付中】</span>';
                ?>
                <a href="vote.php?id=<?= h($poll['id']) ?>" class="poll-link">
                    <div class="card">
                        <h2><?= $status_text ?> <?= h($poll['title']) ?></h2>
                        <div class="meta">終了日時: <?= h($poll['end_time']) ?></div>
                    </div>
                </a>
            <?php endforeach; ?>
        <?php endif; ?>
    </div>
</body>
</html>

---
<?php
// vote.php
require 'db.php';

$poll_id = $_GET['id'] ?? null;
if (!$poll_id) { header("Location: index.php"); exit; }

// スレッド情報の取得
$stmt = $pdo->prepare("SELECT * FROM polls WHERE id = ?");
$stmt->execute([$poll_id]);
$poll = $stmt->fetch(PDO::FETCH_ASSOC);

if (!$poll) { die("スレッドが見つかりません"); }

// 選択肢情報の取得
$stmt = $pdo->prepare("SELECT * FROM options WHERE poll_id = ?");
$stmt->execute([$poll_id]);
$options = $stmt->fetchAll(PDO::FETCH_ASSOC);

// 状態判定
$cookie_name = "voted_poll_" . $poll_id;
$has_voted = isset($_COOKIE[$cookie_name]);
$is_expired = new DateTime() > new DateTime($poll['end_time']);
$show_results = $has_voted || $is_expired;

// 投票処理（POST時）
if ($_SERVER['REQUEST_METHOD'] === 'POST' && !$show_results) {
    $option_id = $_POST['option_id'];
    
    // 投票数を加算
    $stmt = $pdo->prepare("UPDATE options SET votes = votes + 1 WHERE id = ? AND poll_id = ?");
    $stmt->execute([$option_id, $poll_id]);

    // Cookieをセット（有効期限30日）
    setcookie($cookie_name, '1', time() + (86400 * 30), "/");
    
    // 再読み込みして結果を表示
    header("Location: vote.php?id=" . $poll_id);
    exit;
}

// 総投票数の計算（割合表示用）
$total_votes = 0;
foreach ($options as $opt) { $total_votes += $opt['votes']; }
?>
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <title><?= h($poll['title']) ?> - 投票</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="header">
        <h1>投票画面</h1>
        <a href="index.php" class="btn btn-secondary">← 一覧に戻る</a>
    </div>

    <div class="card">
        <h2><?= h($poll['title']) ?></h2>
        <p class="meta">終了期限: <?= h($poll['end_time']) ?></p>
        
        <?php if ($is_expired && !$has_voted): ?>
            <p style="color: red;">※ この投票は終了しました。</p>
        <?php elseif ($has_voted): ?>
            <p style="color: green;">※ 投票済みです。</p>
        <?php endif; ?>

        <hr>

        <?php if ($show_results): ?>
            <h3>現在の投票結果（総数: <?= $total_votes ?>票）</h3>
            <?php foreach ($options as $opt): ?>
                <?php 
                    $percent = $total_votes > 0 ? round(($opt['votes'] / $total_votes) * 100) : 0;
                ?>
                <div style="margin-bottom: 15px;">
                    <div><?= h($opt['name']) ?> (<?= $opt['votes'] ?>票)</div>
                    <div class="result-bar-bg">
                        <div class="result-bar-fill" style="width: <?= $percent ?>%;"></div>
                        <div class="result-text"><?= $percent ?>%</div>
                    </div>
                </div>
            <?php endforeach; ?>

        <?php else: ?>
            <form method="post">
                <?php foreach ($options as $opt): ?>
                    <button type="submit" name="option_id" value="<?= $opt['id'] ?>" class="btn btn-vote">
                        <?= h($opt['name']) ?> に投票する
                    </button>
                <?php endforeach; ?>
            </form>
        <?php endif; ?>
    </div>
</body>
</html>

---
/* style.css */
body { font-family: sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; background-color: #f9f9f9; }
h1, h2 { color: #333; }
.header { display: flex; justify-content: space-between; align-items: center; margin-bottom: 20px; border-bottom: 2px solid #ddd; padding-bottom: 10px; }
.btn { padding: 10px 20px; text-decoration: none; border-radius: 5px; border: none; cursor: pointer; display: inline-block; }
.btn-primary { background-color: #007bff; color: white; }
.btn-secondary { background-color: #6c757d; color: white; margin-left: 10px; }
.btn-vote { background-color: #28a745; color: white; width: 100%; margin-top: 5px; padding: 15px; font-size: 16px; }
.card { background: white; border: 1px solid #ddd; padding: 15px; margin-bottom: 10px; border-radius: 5px; }
.card:hover { box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
.poll-link { text-decoration: none; color: inherit; display: block; }
.meta { font-size: 0.85em; color: #666; margin-top: 5px; }
.form-group { margin-bottom: 15px; }
.form-group label { display: block; margin-bottom: 5px; font-weight: bold; }
.form-control { width: 100%; padding: 10px; box-sizing: border-box; }
.result-bar-bg { background: #e9ecef; border-radius: 5px; height: 25px; margin-top: 5px; position: relative; }
.result-bar-fill { background-color: #007bff; height: 100%; border-radius: 5px; transition: width 0.3s; }
.result-text { position: absolute; left: 10px; top: 2px; font-size: 0.9em; color: #000; font-weight: bold;}
.add-option { margin-top: 5px; font-size: 0.9em; color: #007bff; cursor: pointer; text-decoration: underline; }
"""

messages = [{"role" : "user", "content" : prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # thinking mode True/False
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8,  # For non thinking
    # temperature = 0.6, top_p = 0.95, # For thinking
    top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# バッファオーバーフロー脆弱性のリスクと対策

## 1. バッファオーバーフローの脆弱性の識別
`create.php` と `vote.php` で、`htmlspecialchars()` 関数を使用してユーザー入力をエスケープしているが、`$poll['title']` や `$poll['end_time']` を直接 HTML 内に埋め込むことで XSS（クロスサイトスクリプティング）のリスクが発生します。`h()` 関数を使用してエスケープしているにもかかわらず、`$poll['title']` や `$poll['end_time']` を直接 `h()` 関数でエスケープしていないため、HTML 内に直接埋め込むことで XSS のリスクが発生します。

## 2. リスクの説明
この脆弱性が発生すると、悪意のあるユーザーが JavaScript を含む文字列を入力することで、他のユーザーのブラウザで任意のコードを実行できる可能性があります。これにより、悪意のあるコードを実行することができ、ユーザーの個人情報を取得したり、他のサイトにリダイレクトするなどの悪影響を及ぼす可能性があります。

## 3. 解決策
`$poll['title']` や `$poll['end_time']` を `h()` 関数でエスケープしてから HTML 内に埋め込むことで、XSS のリスクを回避できます。

```php
// vote.php
<h2><?= h($poll['title']) ?></h2>
<p class="meta">終了期限: <?= h($poll['end_time']) ?></p>
```<|im_end|>


## Save LoRA/Model

In [28]:
# save LoRA (path2dir)
model.save_pretrained("./FT-Results/qwen3-14b-rs/")
tokenizer.save_pretrained("./FT-Results/qwen3-14b-rs/")

('./FT-Results/qwen3-14b-rs/tokenizer_config.json',
 './FT-Results/qwen3-14b-rs/special_tokens_map.json',
 './FT-Results/qwen3-14b-rs/chat_template.jinja',
 './FT-Results/qwen3-14b-rs/vocab.json',
 './FT-Results/qwen3-14b-rs/merges.txt',
 './FT-Results/qwen3-14b-rs/added_tokens.json',
 './FT-Results/qwen3-14b-rs/tokenizer.json')

In [29]:
# save ALL-MODEL
model.save_pretrained_merged("./FT-Results/qwen3-14b-rs/", tokenizer, save_method="merged_16bit")

Found HuggingFace hub cache directory: /home/isusers/b223r030p@kochi-u.ac.jp/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 6 files from cache to ./FT-Results/qwen3-14b-rs/.


Unsloth: Merging weights into 16bit: 100%|████████████████████| 6/6 [07:27<00:00, 74.62s/it]


Unsloth: Merge process complete.


In [30]:
# save ALL-MODEL as .gguf
model.save_pretrained_merged("./FT-Results/qwen3-14b-rs/", tokenizer)

Found HuggingFace hub cache directory: /home/isusers/b223r030p@kochi-u.ac.jp/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 6 files from cache to ./FT-Results/qwen3-14b-rs/.


Unsloth: Merging weights into 16bit: 100%|████████████████████| 6/6 [07:20<00:00, 73.45s/it]


Unsloth: Merge process complete.


## Upload LoRA/Model

In [ ]:
# upload
